In [33]:
# Imports
from tensorflow.keras.layers import TextVectorization
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense 
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers


In [42]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]

train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [35]:
vectoriser = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectoriser.adapt(text_ds)

In [36]:
voc = vectoriser.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [37]:
glove_path = "glove.6B.100d.txt"
embeddings_index = {}
with open(glove_path,encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep= " ")
        embeddings_index[word] = coefs

In [38]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

In [39]:
embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)

In [40]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(units=1)(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 100)         1377700   
_________________________________________________________________
conv1d_23 (Conv1D)           (None, None, 128)         12928     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, None, 128)         1651

In [48]:
x_train = vectoriser(np.array([[s] for s in train_samples])).numpy()
x_val = vectoriser(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

In [49]:
model.compile(optimizer=Adam(learning_rate=0.01),loss='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError()])

In [51]:
history = model.fit(x=x_train,y=y_train,batch_size=5,epochs=100)

Epoch 1/100
1125/1125 [==============================] - 9s 7ms/step - loss: 1.4528 - mean_squared_error: 1.4528
Epoch 2/100
1125/1125 [==============================] - 8s 7ms/step - loss: 1.0384 - mean_squared_error: 1.0384
Epoch 3/100
1125/1125 [==============================] - 8s 8ms/step - loss: 0.9224 - mean_squared_error: 0.9224
Epoch 4/100
1125/1125 [==============================] - 8s 7ms/step - loss: 1.0351 - mean_squared_error: 1.0351
Epoch 5/100
1125/1125 [==============================] - 8s 7ms/step - loss: 1.0695 - mean_squared_error: 1.0695
Epoch 6/100
1125/1125 [==============================] - 8s 7ms/step - loss: 1.0640 - mean_squared_error: 1.0640
Epoch 7/100
1125/1125 [==============================] - 8s 7ms/step - loss: 1.0875 - mean_squared_error: 1.0875
Epoch 8/100
1125/1125 [==============================] - 8s 7ms/step - loss: 1.0059 - mean_squared_error: 1.0059
Epoch 9/100
1125/1125 [==============================] - 8s 7ms/step - loss: 1.0187 - mean_squar